In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
from matplotlib.font_manager import FontProperties
import pyscisci.all as pyscisci
from brokenaxes import brokenaxes
from matplotlib import gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import LogLocator
from matplotlib.ticker import MultipleLocator
import matplotlib.ticker as ticker
from scipy.stats import norm, ks_2samp
from collections import defaultdict
EF = FontProperties(family = 'Arial')

%matplotlib inline

In [14]:
Publication = r'/Users/mingzezhang/Downloads/0-Publications.csv'
Pdf = pd.read_csv(Publication)
print(Pdf .shape)
del Pdf['Unnamed: 0']
Pdf .columns

(993998, 5)


Index(['work_id', 'PublishedYear', 'DocumentType', 'Journal'], dtype='object')

In [15]:
Pdf = Pdf[Pdf['PublishedYear']<=2023]
# Article Type
Pdf = Pdf[Pdf['DocumentType']=='article']
print(Pdf .shape)

(928321, 4)


In [16]:
Pdf['Journal'].value_counts()

Journal
Nature                                             412342
Science                                            363947
Proceedings of the National Academy of Sciences    152032
Name: count, dtype: int64

In [4]:
Works_authorship = r'/Users/mingzezhang/Downloads/1-Work_Authorship.csv'
WAdf = pd.read_csv(Works_authorship)
del WAdf['Unnamed: 0']
print(WAdf .shape)
WAdf .columns

(3419221, 4)


Index(['work_id', 'author_position', 'author_id', 'institution_id'], dtype='object')

In [ ]:
WAdf = WAdf .drop_duplicates(subset = ['work_id','author_id'])
print(WAdf .shape)

WAdf['institution_id'] = WAdf['institution_id'].fillna(0)
WAdf_na = WAdf[WAdf['institution_id']==0]
WAdf = WAdf[~WAdf['work_id'].isin(WAdf_na['work_id'].drop_duplicates())]
print(WAdf .shape)

(2484365, 4)
(1653692, 4)


In [6]:
Pdf = Pdf[Pdf['work_id'].isin(WAdf['work_id'].drop_duplicates())]
print(Pdf .shape)

(407120, 4)


In [8]:
WAdf = WAdf[WAdf['work_id'].isin(Pdf['work_id'].drop_duplicates())]
print(WAdf .shape)

(1509324, 4)


In [9]:
WAdf = WAdf .merge(Pdf[['work_id','Journal']],
                   how = 'left', left_on = 'work_id', right_on = 'work_id')

In [10]:
WAdf .columns

Index(['work_id', 'author_position', 'author_id', 'institution_id', 'Journal'], dtype='object')

In [12]:
Inst = r'/Users/mingzezhang/Downloads/Data_OpenAlex_202501/institutions_geo.csv.gz'
Inst_columns = ['institution_id','country_code']
chunk_size = 50000000
Instdf = pd.DataFrame()
for chunk in pd.read_csv(Inst, usecols = Inst_columns, chunksize = chunk_size, compression = 'gzip'):
    chunk = chunk[chunk['institution_id'].isin(WAdf['institution_id'].drop_duplicates())]
    print(chunk .shape)
    Instdf = pd.concat([Instdf, chunk], ignore_index = True)
    del chunk
Instdf .columns

WAdf = WAdf .merge(Instdf[['institution_id','country_code']],
                   how = 'left', left_on = 'institution_id', right_on = 'institution_id')
print(WAdf .shape)

(20723, 2)
(1509324, 6)


In [13]:
WAdf .to_csv(r'/Users/mingzezhang/Downloads/BasicDataforDistribution.csv')